# Dog Breeds

In [257]:
# import useful packages
import pandas as pd
import numpy as np
from numpy import argmax
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import History, TensorBoard

In [259]:
# callbacks
history = History()
tb = TensorBoard()

In [236]:
labels_df = pd.read_csv('/mnt/DataDisk/jodahr/data/Dogs/labels.csv')

In [237]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10222 entries, 0 to 10221
Data columns (total 2 columns):
id       10222 non-null object
breed    10222 non-null object
dtypes: object(2)
memory usage: 159.8+ KB


In [238]:
labels_df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


## Prepare Training Data

In [239]:
import sys, os, subprocess
from shutil import copyfile
import shutil

In [240]:
# make subdirs for dog breeds
for breed in breeds:
    os.makedirs('/mnt/DataDisk/jodahr/data/Dogs/training/' + breed, exist_ok=True)
    os.makedirs('/mnt/DataDisk/jodahr/data/Dogs/validation/' + breed, exist_ok=True)

In [241]:
# image path
path = '/mnt/DataDisk/jodahr/data/Dogs/train/'

In [131]:
from sklearn.cross_validation import train_test_split
train, test = train_test_split(labels_df, test_size=0.3)

In [134]:
for index, row in train.iterrows():
    filename = row['id'] + '.jpg'
    src = path + filename
    target = '/mnt/DataDisk/jodahr/data/Dogs/training/' + row['breed']
    shutil.copy(src=src, dst=target)

In [135]:
for index, row in test.iterrows():
    filename = row['id'] + '.jpg'
    src = path + filename
    target = '/mnt/DataDisk/jodahr/data/Dogs/validation/' + row['breed']
    shutil.copy(src=src, dst=target)

## Keras Model

In [247]:
# dimensions of our images.
img_width, img_height = 250, 250

train_data_dir = '/mnt/DataDisk/jodahr/data/Dogs/training/'
validation_data_dir = '/mnt/DataDisk/jodahr/data/Dogs/validation/'
nb_train_samples = 7155
nb_validation_samples = 3067
epochs = 150
batch_size = 32

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [248]:
# model topology

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(120))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [249]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [250]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 7155 images belonging to 120 classes.
Found 3067 images belonging to 120 classes.


In [251]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 248, 248, 64)      1792      
_________________________________________________________________
activation_66 (Activation)   (None, 248, 248, 64)      0         
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 124, 124, 64)      0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 122, 122, 64)      36928     
_________________________________________________________________
activation_67 (Activation)   (None, 122, 122, 64)      0         
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 61, 61, 64)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 59, 59, 64)        36928     
__________

In [260]:
with tf.device('/cpu:0'):
    model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks = [history, tb])

Epoch 1/150
223/223 [==============================] - 87s 391ms/step - loss: 4.7752 - acc: 0.0121 - val_loss: 4.7776 - val_acc: 0.0145
Epoch 2/150
223/223 [==============================] - 87s 390ms/step - loss: 4.7480 - acc: 0.0185 - val_loss: 4.7033 - val_acc: 0.0204
Epoch 3/150
223/223 [==============================] - 87s 388ms/step - loss: 4.6822 - acc: 0.0198 - val_loss: 4.6393 - val_acc: 0.0234
Epoch 4/150
223/223 [==============================] - 87s 390ms/step - loss: 4.6348 - acc: 0.0241 - val_loss: 4.5664 - val_acc: 0.0283
Epoch 5/150
223/223 [==============================] - 86s 387ms/step - loss: 4.5756 - acc: 0.0304 - val_loss: 4.5290 - val_acc: 0.0339
Epoch 6/150
223/223 [==============================] - 86s 387ms/step - loss: 4.5443 - acc: 0.0300 - val_loss: 4.5034 - val_acc: 0.0359
Epoch 7/150
223/223 [==============================] - 86s 387ms/step - loss: 4.5066 - acc: 0.0370 - val_loss: 4.4854 - val_acc: 0.0349
Epoch 8/150
223/223 [===========================

Epoch 61/150
223/223 [==============================] - 84s 377ms/step - loss: 3.2233 - acc: 0.2146 - val_loss: 4.3385 - val_acc: 0.0908
Epoch 62/150
223/223 [==============================] - 84s 377ms/step - loss: 3.1757 - acc: 0.2075 - val_loss: 4.3753 - val_acc: 0.0941
Epoch 63/150
223/223 [==============================] - 84s 376ms/step - loss: 3.2067 - acc: 0.2076 - val_loss: 4.3188 - val_acc: 0.0921
Epoch 64/150
223/223 [==============================] - 85s 380ms/step - loss: 3.1696 - acc: 0.2098 - val_loss: 4.3478 - val_acc: 0.0888
Epoch 65/150
223/223 [==============================] - 84s 376ms/step - loss: 3.1615 - acc: 0.2205 - val_loss: 4.4468 - val_acc: 0.0941
Epoch 66/150
223/223 [==============================] - 83s 374ms/step - loss: 3.1317 - acc: 0.2204 - val_loss: 4.3637 - val_acc: 0.0859
Epoch 67/150
223/223 [==============================] - 84s 376ms/step - loss: 3.1127 - acc: 0.2224 - val_loss: 4.4020 - val_acc: 0.0895
Epoch 68/150
223/223 [===================

223/223 [==============================] - 84s 377ms/step - loss: 2.6636 - acc: 0.3063 - val_loss: 4.8605 - val_acc: 0.0934
Epoch 121/150
223/223 [==============================] - 84s 375ms/step - loss: 2.6592 - acc: 0.3089 - val_loss: 4.8163 - val_acc: 0.0872
Epoch 122/150
223/223 [==============================] - 84s 378ms/step - loss: 2.6406 - acc: 0.3088 - val_loss: 4.7712 - val_acc: 0.0931
Epoch 123/150
223/223 [==============================] - 85s 379ms/step - loss: 2.6529 - acc: 0.3155 - val_loss: 4.7922 - val_acc: 0.0918
Epoch 124/150
223/223 [==============================] - 84s 376ms/step - loss: 2.6413 - acc: 0.3087 - val_loss: 4.8277 - val_acc: 0.0895
Epoch 125/150
223/223 [==============================] - 84s 377ms/step - loss: 2.6232 - acc: 0.3169 - val_loss: 4.8609 - val_acc: 0.0928
Epoch 126/150
223/223 [==============================] - 84s 378ms/step - loss: 2.6275 - acc: 0.3177 - val_loss: 4.8539 - val_acc: 0.0947
Epoch 127/150
223/223 [=========================

In [256]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9546524367860712878, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 9633792
 locality {
   bus_id: 1
 }
 incarnation: 1049322012252770017
 physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:0c:00.0"]